<a href="https://colab.research.google.com/github/vidosuba/slovak-NER/blob/main/other/trankit_ner_wikiann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 11.3MB/s 
     |████████████████████████████████| 901kB 36.3MB/s 
     |████████████████████████████████| 3.3MB 36.8MB/s 


In [ ]:
!pip install trankit

     |████████████████████████████████| 778kB 10.6MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 1.9MB 37.8MB/s 
     |████████████████████████████████| 1.2MB 35.8MB/s 
  Created wheel for langid: filename=langid-1.1.6-cp37-none-any.whl size=1941191 sha256=53d91201239c6d9416cd950f4df3ffe3bf68937598627a21f11cc159c974c40c
  Stored in directory: /root/.cache/pip/wheels/29/bc/61/50a93be85d1afe9436c3dc61f38da8ad7b637a38af4824e86e
Successfully built langid
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


### Prepare the data

In [ ]:
!mkdir -p data
!cd data

!wget https://raw.githubusercontent.com/vidosuba/slovak-NER/main/wikiann/cleaned_data/dev_cleaned.txt -P data
!wget https://raw.githubusercontent.com/vidosuba/slovak-NER/main/wikiann/cleaned_data/dev_cleaned.txt -P data
!wget https://raw.githubusercontent.com/vidosuba/slovak-NER/main/wikiann/cleaned_data/dev_cleaned.txt -P data

--2021-04-28 13:50:17--  https://raw.githubusercontent.com/vidosuba/slovak-NER/main/wikiann/cleaned_data/dev_cleaned.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 690940 (675K) [text/plain]
Saving to: ‘data/dev_cleaned.txt’

dev_cleaned.txt     100%[===================>] 674.75K  --.-KB/s    in 0.01s   

2021-04-28 13:50:17 (44.0 MB/s) - ‘data/dev_cleaned.txt’ saved [690940/690940]

--2021-04-28 13:50:17--  https://raw.githubusercontent.com/vidosuba/slovak-NER/main/wikiann/cleaned_data/dev_cleaned.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting re

## Run the training loop

In [ ]:
import trankit

# initialize a trainer for the task
trainer = trankit.TPipeline(
    training_config={
      'max_epoch': 1,
      'category': 'customized-ner',  # pipeline category
      'task': 'ner', # task name
      'save_dir': './save_dir_filtered', # directory to save the trained model
      'train_bio_fpath': './data/train_cleaned.txt', # training data in BIO format
      'dev_bio_fpath': './data/dev_cleaned.txt' # training data in BIO format
    }
)

# start training
trainer.train()

Setting up training config...



Skipped 0 over-length examples
Loaded 20000 examples
Skipped 0 over-length examples
Loaded 10000 examples


Train 0:   0%|                                    | 0/1250 [00:00<?, ?it/s]

******************************
NER: Epoch: 0


dev 0: 100%|█████████████████████████████| 625/625 [01:01<00:00, 10.18it/s]


Saving adapter weights to ... ./save_dir_filtered/customized-ner/customized-ner.ner.mdl (9.96 MB)
------------------------------
Best dev F1 score: epoch 0, F1: 68.17
Training done!


In [ ]:
from trankit.iterators.ner_iterators import NERDataset
test_set = NERDataset(
    config=trainer._config,
    bio_fpath='./data/test_cleaned.txt',
    evaluate=True
)
test_set.numberize()
test_batch_num = len(test_set) // trainer._config.batch_size + (len(test_set) % trainer._config.batch_size != 0)
result = trainer._eval_ner(data_set=test_set, batch_num=test_batch_num,
                           name='test', epoch=-1)

test -1:   0%|                             | 2/625 [00:00<00:44, 13.97it/s]

Skipped 0 over-length examples
Loaded 10000 examples


test -1: 100%|███████████████████████████| 625/625 [01:01<00:00, 10.24it/s]


In [ ]:
print(result)

{'p': 66.11087953313881, 'r': 71.69981916817359, 'f1': 68.79201908479723}


## Running the trained model on the `test` set

In [ ]:
!wget -q http://nlp.uoregon.edu/download/trankit/slovak.zip

In [ ]:
%%shell
unzip -o slovak.zip

Archive:  slovak.zip
 extracting: slovak.downloaded       
  inflating: slovak_lemmatizer.pt    
  inflating: slovak.tagger.mdl       
  inflating: slovak.tokenizer.mdl    
  inflating: slovak.vocabs.json      


In [ ]:
!cp -a slovak.tagger.mdl  ./save_dir_filtered/customized-ner/customized-ner.tagger.mdl
!cp -a slovak.vocabs.json  ./save_dir_filtered/customized-ner/customized-ner.vocabs.json
!cp -a slovak_lemmatizer.pt  ./save_dir_filtered/customized-ner/customized-ner_lemmatizer.pt
!cp -a slovak.tokenizer.mdl  ./save_dir_filtered/customized-ner/customized-ner.tokenizer.mdl

In [ ]:
trankit.verify_customized_pipeline(
    category='customized-ner', # pipeline category
    save_dir='./save_dir_filtered' # directory used for saving models in previous steps
)

Customized pipeline is ready to use!
It can be initialized as follows:
-----------------------------------
from trankit import Pipeline
p = Pipeline(lang='customized-ner', cache_dir='./save_dir_filtered')


## Using the pretrained model in a Pipeline

In [ ]:
from trankit import Pipeline

In [ ]:
p = Pipeline(lang='customized-ner', cache_dir='./save_dir_filtered')


http://nlp.uoregon.edu/download/trankit/v1.0.0/xlm-roberta-base/customized-ner.zip


Downloading: 100%|██████████| 196/196 [00:00<00:00, 63.8kiB/s]


BadZipFile: ignored

In [ ]:
p.ner('''Vlani súd v Bratislave vymeral Marošovi Deákovi 25 rokov väzenia za rôzne zločiny''', is_sent=True)

NameError: ignored